In [1]:
import pandas as pd
import numpy as np

In [70]:
LIFT_reg = pd.read_csv('/Users/albertsun/Projects/data/LIFT_registry_2022-04-01.csv')
LIFT_reg['stage'] = LIFT_reg['card_id'].str[-1:]
LIFT_reg['id'] = LIFT_reg['card_id'].str[:-2]
LIFT_reg['FIPS'] = LIFT_reg['FIPS'].astype(str).replace('\.0', '', regex=True)
uq_reg = LIFT_reg.sort_values('stage', ascending=False).drop_duplicates(['id'])
uq_reg = uq_reg[uq_reg.FIPS != 'nan'] # remove nan
uq_reg

,Age,RaceDesc,LanguageSpoken,Expiration,DateIssued,CardIssuingAgency,FIPS,Initial Load,Study Card,duplicate,card_id,stage,id
13395,40,Asian,English,8/31/2023,2021-09-28,CCS,530330092002,Subsidized Annual Pass,NaN,False,61080-8,8,61080
4794,41,White,English,8/31/2023,2021-09-28,CCS,530330017012,Subsidized Annual Pass,NaN,False,18516-6,6,18516
39267,66,Black or African American,English,8/31/2024,2021-10-13,King County Public Health,530330040001,Subsidized Annual Pass,NaN,False,126848-6,6,126848
10076,44,Black or African American,English,8/31/2023,2021-09-29,CCS,530330095004,Subsidized Annual Pass,NaN,False,43232-6,6,43232
5499,61,White,English,8/31/2023,2021-09-28,CCS,530330082001,Subsidized Annual Pass,NaN,False,23028-6,6,23028
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45465,27,White,English,2/28/2021,2018-04-18,King County Public Health,530330224004,NaN,NaN,False,137458-1,1,137458
45464,37,White,English,11/30/2020,2018-04-18,CSO Auburn,530330252001,10,NaN,False,137456-1,1,137456
45463,26,White,English,8/30/2019,2018-04-18,WithinReach,530610519151,NaN,NaN,False,137454-1,1,137454
45462,35,White,English,2/28/2021,2018-04-18,King County Public Health,530330067003,NaN,NaN,False,137452-1,1,137452


In [63]:
unequal = pd.read_csv('/Users/albertsun/Projects/data/job_access_gap.csv')
unequal = unequal.iloc[:,:7]
unequal = unequal[unequal.MSA=='Seattle']
unequal['GEOID'] = unequal['GEOID'].astype(str)
unequal

,GEOID,OverallJobAccess,lowwagelaborforce,Access30Transit,Access30Transit_ts,MSA,spatialmismatch
0,530610518023,1.530868,413,0.063454,0.030801,Seattle,0.073675
1,530530634006,1.104112,186,0.015915,0.000264,Seattle,0.039919
2,530330219042,1.485395,174,0.080578,0.075943,Seattle,0.031712
3,530330219044,1.194115,200,0.067042,0.075943,Seattle,0.041396
4,530330219051,2.029325,283,0.086625,0.060658,Seattle,0.038509
...,...,...,...,...,...,...,...
2478,530330293035,1.236520,364,0.000930,0.000930,Seattle,0.074030
2479,530330300046,0.842214,479,0.035533,0.030762,Seattle,0.113454
2480,530330303081,0.419282,263,0.013390,0.005742,Seattle,0.071736
2481,530330323272,0.565111,125,0.004398,0.004398,Seattle,0.032548


In [89]:
merged = uq_reg.merge(unequal, left_on='FIPS', right_on='GEOID', how='inner')
cols = ['OverallJobAccess', 'lowwagelaborforce', 'Access30Transit', 'Access30Transit_ts', 'spatialmismatch']
for col in cols: 
    merged[col + '_quantile_4q'] = pd.qcut(merged[col], q=4)
    merged[col + '_quantileNum_4q'] = pd.qcut(merged[col], q=4, labels=False)
    
for col in cols: 
    merged[col + '_quantile_10q'] = pd.qcut(merged[col], q=10)
    merged[col + '_quantileNum_10q'] = pd.qcut(merged[col], q=10, labels=False)
    
merged

,Age,RaceDesc,LanguageSpoken,Expiration,DateIssued,CardIssuingAgency,FIPS,Initial Load,Study Card,duplicate,...,OverallJobAccess_quantile_10q,OverallJobAccess_quantileNum_10q,lowwagelaborforce_quantile_10q,lowwagelaborforce_quantileNum_10q,Access30Transit_quantile_10q,Access30Transit_quantileNum_10q,Access30Transit_ts_quantile_10q,Access30Transit_ts_quantileNum_10q,spatialmismatch_quantile_10q,spatialmismatch_quantileNum_10q
0,40,Asian,English,8/31/2023,2021-09-28,CCS,530330092002,Subsidized Annual Pass,NaN,False,...,"(1.972, 2.246]",8,"(543.0, 708.0]",8,"(1.275, 3.047]",9,"(0.637, 3.047]",9,"(0.0765, 0.0833]",5
1,51,Black or African American,English,8/31/2023,2021-10-19,CCS,530330092002,Subsidized Annual Pass,NaN,False,...,"(1.972, 2.246]",8,"(543.0, 708.0]",8,"(1.275, 3.047]",9,"(0.637, 3.047]",9,"(0.0765, 0.0833]",5
2,42,Black or African American,English,4/30/2023,2021-04-29,HS Jail Health Services BA6586,530330092002,NaN,NaN,False,...,"(1.972, 2.246]",8,"(543.0, 708.0]",8,"(1.275, 3.047]",9,"(0.637, 3.047]",9,"(0.0765, 0.0833]",5
3,59,Black or African American,English,8/31/2023,2021-11-23,King County Public Health,530330092002,Subsidized Annual Pass,NaN,False,...,"(1.972, 2.246]",8,"(543.0, 708.0]",8,"(1.275, 3.047]",9,"(0.637, 3.047]",9,"(0.0765, 0.0833]",5
4,44,Hispanic or Latino,English,8/31/2023,2021-11-10,CCS,530330092002,Subsidized Annual Pass,NaN,False,...,"(1.972, 2.246]",8,"(543.0, 708.0]",8,"(1.275, 3.047]",9,"(0.637, 3.047]",9,"(0.0765, 0.0833]",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93655,36,Asian,English,2/28/2021,2018-05-15,King County Public Health,530330322122,NaN,NaN,False,...,"(0.526, 0.73]",1,"(1.999, 236.0]",0,"(0.00344, 0.0212]",1,"(0.00118, 0.00441]",1,"(0.0504, 0.0594]",2
93656,36,Other,English,2/28/2021,2018-05-15,King County Public Health,530330322122,NaN,NaN,False,...,"(0.526, 0.73]",1,"(1.999, 236.0]",0,"(0.00344, 0.0212]",1,"(0.00118, 0.00441]",1,"(0.0504, 0.0594]",2
93657,26,Multi-Racial,English,7/31/2020,2018-06-11,CCS BA3628,530330322122,Monthly Pass,NaN,False,...,"(0.526, 0.73]",1,"(1.999, 236.0]",0,"(0.00344, 0.0212]",1,"(0.00118, 0.00441]",1,"(0.0504, 0.0594]",2
93658,58,White,English,3/31/2020,2018-06-13,King County Public Health,530530724083,NaN,NaN,False,...,"(0.92, 1.098]",3,"(236.0, 281.0]",1,"(0.00344, 0.0212]",1,"(0.00441, 0.02]",2,"(0.0504, 0.0594]",2


In [94]:
for col in cols: 
    print(col, merged[col + '_quantile_4q'].unique())

OverallJobAccess [(1.81, 3.632], (1.26, 1.81], (0.0635, 0.81], (0.81, 1.26]]
Categories (4, interval[float64, right]): [(0.0635, 0.81] < (0.81, 1.26] < (1.26, 1.81] < (1.81, 3.632]]
lowwagelaborforce [(504.0, 1252.0], (1.999, 297.0], (297.0, 385.0], (385.0, 504.0]]
Categories (4, interval[float64, right]): [(1.999, 297.0] < (297.0, 385.0] < (385.0, 504.0] < (504.0, 1252.0]]
Access30Transit [(0.436, 3.047], (0.0919, 0.436], (0.0312, 0.0919], (-0.001, 0.0312]]
Categories (4, interval[float64, right]): [(-0.001, 0.0312] < (0.0312, 0.0919] < (0.0919, 0.436] < (0.436, 3.047]]
Access30Transit_ts [(0.258, 3.047], (0.0654, 0.258], (-0.001, 0.00981], (0.00981, 0.0654]]
Categories (4, interval[float64, right]): [(-0.001, 0.00981] < (0.00981, 0.0654] < (0.0654, 0.258] < (0.258, 3.047]]
spatialmismatch [(0.0765, 0.0994], (-0.001, 0.0564], (0.0564, 0.0765], (0.0994, 0.365]]
Categories (4, interval[float64, right]): [(-0.001, 0.0564] < (0.0564, 0.0765] < (0.0765, 0.0994] < (0.0994, 0.365]]


In [95]:
for col in cols: 
    print(col, merged[col + '_quantile_10q'].unique())

OverallJobAccess [(1.972, 2.246], (1.462, 1.677], (0.0635, 0.526], (1.26, 1.462], (1.677, 1.972], (1.098, 1.26], (2.246, 3.632], (0.73, 0.92], (0.526, 0.73], (0.92, 1.098]]
Categories (10, interval[float64, right]): [(0.0635, 0.526] < (0.526, 0.73] < (0.73, 0.92] < (0.92, 1.098] ... (1.462, 1.677] < (1.677, 1.972] < (1.972, 2.246] < (2.246, 3.632]]
lowwagelaborforce [(543.0, 708.0], (236.0, 281.0], (281.0, 314.0], (385.0, 429.0], (429.0, 481.0], (1.999, 236.0], (481.0, 543.0], (355.0, 385.0], (708.0, 1252.0], (314.0, 355.0]]
Categories (10, interval[float64, right]): [(1.999, 236.0] < (236.0, 281.0] < (281.0, 314.0] < (314.0, 355.0] ... (429.0, 481.0] < (481.0, 543.0] < (543.0, 708.0] < (708.0, 1252.0]]
Access30Transit [(1.275, 3.047], (0.0919, 0.142], (0.142, 0.262], (0.262, 0.697], (0.697, 1.275], (0.0387, 0.0625], (0.0625, 0.0919], (0.00344, 0.0212], (-0.001, 0.00344], (0.0212, 0.0387]]
Categories (10, interval[float64, right]): [(-0.001, 0.00344] < (0.00344, 0.0212] < (0.0212, 0.03

In [88]:
merged.to_csv('LIFT_registry_UCMerged_new.csv')

98228 people in the lift registry, 94133 people in the lift registry with a FIPS code, 93660 people with a FIPS in the unequal commute dataset.